In [174]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [175]:
class Applicant:
    def __init__(self, name, preferences_of_school):
        self.name = name
        self.preferences_of_school = preferences_of_school
        self.proposal_index = 0  # Индекс в списке предпочтений, куда была сделана предложение
        self.partner = None  # Проект, с которым студент образует пару

    def make_proposal(self):
        # Метод для сделать предложение выбранным проектом из списка предпочтений
        preference_of_school = self.preferences_of_school[self.proposal_index]
        self.proposal_index += 1
        return preference_of_school

    def receive_partner(self, partner):
        # Метод для добавления проекта
        self.partner = partner

    def is_free(self):
        # Метод, возвращающий True, если студент свободен и еще не образовал пару
        return self.partner is None


class Project:
    def __init__(self, name, capacity, preferences_of_students):
        self.name = name
        self.capacity = capacity  # количество
        self.preferences_of_students = preferences_of_students  # предпочтения проекта
        self.applicants = []

    def add_applicant(self, applicant):
        # Метод для добавления студента в список принятых проектом
        self.applicants.append(applicant)

    def remove_applicant(self, applicant):
        # Метод для удаления студента из списка принятых проектом
        self.applicants.remove(applicant)

    def has_capacity(self):
        # Метод, возвращающий True, если в проекте есть свободные места
        return len(self.applicants) < self.capacity

    def prefers_applicant(self, applicant):
        # Метод, проверяющий, предпочитает ли проекта данный студент
        return applicant in self.preferences_of_students[:self.capacity]

    def get_least_preferred_applicant(self):
        # Метод, возвращающий наименее предпочитаемого студента в проекте
        return self.applicants[-1]


def gale_shapley(applicants, schools):
    while True:
        free_applicants = [applicant for applicant in applicants if applicant.is_free()]

        if not free_applicants:
            break

        for applicant in free_applicants:
            #print(applicant.name)
            school_name = str(applicant.make_proposal())
            school = schools[school_name]

            if school.has_capacity():
                school.add_applicant(applicant)
                applicant.receive_partner(school)
                applicant.priority = applicant.preferences_of_school.index(school.name) + 1
        else:
            least_preferred_applicant = school.get_least_preferred_applicant()

            if school.prefers_applicant(applicant):
                school.remove_applicant(least_preferred_applicant)
                least_preferred_applicant.receive_partner(None)

                school.add_applicant(applicant)
                applicant.receive_partner(school)

    matches = [[applicant, applicant.partner] for applicant in applicants]
    return matches

In [202]:
ratings = pd.read_excel('Отбор ШМЯ 2023 - финал (1) (3).xlsx')
ratings = ratings[ratings['Вердикт Лены'] == 'Да']
ratings = ratings[ratings['В каком хабе будет во втором этапе'] == 'Москва']
ratings['Балл за зум-собеседование (макс 10)'] = ratings['Балл за зум-собеседование (макс 10)'].replace('нет оценки', np.nan)

In [203]:
avg_zoom_scores = ratings['Балл за зум-собеседование (макс 10)'].mean()

In [204]:
ratings['Балл за зум-собеседование (макс 10)'] = ratings['Балл за зум-собеседование (макс 10)'].fillna(avg_zoom_scores)
ratings['Итоговая сумма'] = ratings['Балл за зум-собеседование (макс 10)'] + ratings['Балл за VCV (макс 10)'] + ratings['Оценка за мотивацию (макс 3)'] + ratings['Сумма за кейсы (макс 10)']
ratings['фамилия + имя'] = ratings['Фамилия кандидата'].str.strip().str.lower() +' '+ ratings['Имя кандидата'].str.strip().str.lower()

In [205]:
ratings = ratings[['фамилия + имя', 'Итоговая сумма']]

In [206]:
ratings = ratings.sort_values(by='Итоговая сумма', ascending=False)

In [207]:
ratings

,фамилия + имя,Итоговая сумма
76,хажиев роберт,27.00000
7,мировский слава,27.00000
1,зубайдуллин ринат,26.00000
10,алейникова дарья,26.00000
57,овчинников григорий,26.00000
52,мурзаева евгения,26.00000
41,крылов артем,26.00000
32,иванченко алена,25.00000
68,тагиев станислав,25.00000
16,берхольц виктория,25.00000


In [208]:
students_choose_projects = pd.read_excel('2023-06-12 ShMIa 2023 vybor proekta.xlsx')
students_choose_projects = students_choose_projects[(students_choose_projects['Твой хаб'] == 'Москва') & (students_choose_projects['Направление в ШМЯ'] == 'Управление проектами и продуктами')]
students_choose_projects['фамилия + имя'] = students_choose_projects['Фамилия'].str.strip().str.lower() + ' '+ students_choose_projects['Имя'].str.strip().str.lower()
students_choose_projects = students_choose_projects.drop(['Фамилия', 'Имя', 'Твой хаб', 'Направление в ШМЯ'], axis=1)
students_choose_projects = students_choose_projects.astype(str)

In [209]:
students_choose_projects

,1 приоритет,2 приоритет,3 приоритет,4 приоритет,5 приоритет,6 приоритет,7 приоритет,8 приоритет,9 приоритет,10 приоритет,11 приоритет,12 приоритет,13 приоритет,14 приоритет,15 приоритет,фамилия + имя
8,19,12,25,32,26,33,20,50,28,18,24,48,29,22,43,хури полина
16,25,12,22,37,50,34-35,23,24,14,40,31,56-57,38,30,51,исламуратов марат
30,14,13,49,43,48,31,19,50,56,57,12,22,50,40,47,алейникова дарья
33,20,14,25,12,51,36,37,47,34-35,34-35,48,33,26,50,19,ревнивцев андрей
40,36,14,33,15,45-46,24,12,38,20,22,23,13,31,28,42,бурзунов данил
44,48,14,13,45-46,33,19,49,44,32,29,12,22,15,26,28,головач максим
46,45-46,19,13,14,45-46,31,47,24,23,20,55,44,28,27,49,нестерович светлана
47,48,19,32,14,33,12,45-46,40,38,31,47,34-35,20,28,25,мурзаева евгения
48,19,22,33,31,37,45-46,47,48,34-35,51,14,25,26,38,32,овчинников григорий
50,51,22,45-46,32,13,31,17,18,47,37,36,19,20,28,26,целищев максим


In [210]:
cols = ['1 приоритет', '2 приоритет', '3 приоритет', '4 приоритет', '5 приоритет', '6 приоритет', '7 приоритет', '8 приоритет', '9 приоритет', '10 приоритет', '11 приоритет', '12 приоритет', '13 приоритет', '14 приоритет', '15 приоритет']
lst_of_all_projects = pd.concat([students_choose_projects[col] for col in cols]).unique()
#lst_of_all_projects

In [211]:
applicants = []
for _, row in students_choose_projects.iterrows():
    name = row['фамилия + имя']  # Использование столбца 'Фамилия + имя' из students_choose_projects
    projects = []
    for i in range(1, 16):  # Итерация по столбцам с приоритетами ('1 приоритет', '2 приоритет', ..., '15 приоритет')
        project = row[str(i) + ' приоритет']  # Получение значения приоритета проекта
        projects.append(project)

    remaining_projects = set(lst_of_all_projects) - set(projects)
    projects += list(remaining_projects)

    applicant = Applicant(name, projects)
    applicants.append(applicant)

In [212]:
projects = {}
for project_number in lst_of_all_projects:
    name = str(project_number)
    capacity = 2 if name == '48' else 1  # Устанавливаем capacity=2 для проекта с номером '48', иначе capacity=1
    students = list(ratings['фамилия + имя'])  # Заполняем список студентов из столбца 'фамилия + имя' в DataFrame ratings
    project = Project(name, capacity, students)
    projects[name] = project

In [213]:
matches = gale_shapley(applicants, projects)

#[print(elem[0].name, elem[1].name) for elem in matches]

In [217]:
[print(elem[0].name, elem[0].partner) for elem in matches]

хури полина <__main__.Project object at 0x000001AC58822DA0>
исламуратов марат <__main__.Project object at 0x000001AC58823940>
алейникова дарья <__main__.Project object at 0x000001AC588218A0>
ревнивцев андрей <__main__.Project object at 0x000001AC58822320>
бурзунов данил <__main__.Project object at 0x000001AC58822B60>
головач максим <__main__.Project object at 0x000001AC58C38190>
нестерович светлана <__main__.Project object at 0x000001AC58C39300>
мурзаева евгения <__main__.Project object at 0x000001AC58C38190>
овчинников григорий <__main__.Project object at 0x000001AC58C38FA0>
целищев максим <__main__.Project object at 0x000001AC58C3B430>
крылов артем <__main__.Project object at 0x000001AC58C3A230>
абазов руслан <__main__.Project object at 0x000001AC58C3AB60>
анисимов максим <__main__.Project object at 0x000001AC58C3AC80>
закиров дмитрий <__main__.Project object at 0x000001AC58C3A350>
иванов сергей <__main__.Project object at 0x000001AC58C3A980>
антонова дарья <__main__.Project object a

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [218]:
[print(elem[0], elem[1]) for elem in matches]

<__main__.Applicant object at 0x000001AC5764DDB0> <__main__.Project object at 0x000001AC58822DA0>
<__main__.Applicant object at 0x000001AC58822CB0> <__main__.Project object at 0x000001AC58823940>
<__main__.Applicant object at 0x000001AC5764C580> <__main__.Project object at 0x000001AC588218A0>
<__main__.Applicant object at 0x000001AC5764C640> <__main__.Project object at 0x000001AC58822320>
<__main__.Applicant object at 0x000001AC5764E470> <__main__.Project object at 0x000001AC58822B60>
<__main__.Applicant object at 0x000001AC5764C6A0> <__main__.Project object at 0x000001AC58C38190>
<__main__.Applicant object at 0x000001AC573704F0> <__main__.Project object at 0x000001AC58C39300>
<__main__.Applicant object at 0x000001AC57371B10> <__main__.Project object at 0x000001AC58C38190>
<__main__.Applicant object at 0x000001AC5764C1F0> <__main__.Project object at 0x000001AC58C38FA0>
<__main__.Applicant object at 0x000001AC5764DA20> <__main__.Project object at 0x000001AC58C3B430>
<__main__.Applicant 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [190]:
ratings

,фамилия + имя,Итоговая сумма
76,хажиев роберт,27.00000
7,мировский слава,27.00000
1,зубайдуллин ринат,26.00000
10,алейникова дарья,26.00000
57,овчинников григорий,26.00000
52,мурзаева евгения,26.00000
41,крылов артем,26.00000
32,иванченко алена,25.00000
68,тагиев станислав,25.00000
16,берхольц виктория,25.00000


In [219]:
def metrics(matches):
    ind_of_recieved_projects = [matches[i][0].preferences_of_school.index(matches[i][1].name) + 1 for i in
                                range(len(matches))]

    ind_of_recieved_students_by_project = defaultdict(list)
    for elem in matches:
        ind_of_recieved_students_by_project[elem[1].name].append(
            elem[1].preferences_of_students.index(elem[0].name) + 1)
    return f'среднее место проекта у студентов: {np.mean(ind_of_recieved_projects):.3f}, среднее место студентов у проектов: {np.mean(list(map(np.mean, ind_of_recieved_students_by_project.values()))):.3f}'


metrics(matches)

'среднее место проекта у студентов: 5.364, среднее место студентов у проектов: 23.151'

In [220]:
raspredelenie = pd.read_excel('Студенты по локациям 2023.xlsx', sheet_name='Распределение студентов')
df = pd.DataFrame(raspredelenie, columns=['Unnamed: 3', 'Unnamed: 7'])
df['фамилия + имя'] = df['Unnamed: 3'].str.lower()
df = df.drop('Unnamed: 3', axis=1)
df = df.rename(columns={'Unnamed: 7': 'проект'})

df['Word Count'] = df['фамилия + имя'].str.split().str.len()
df_first = df[df['Word Count'] == 2].drop('Word Count', axis=1)
##
df = pd.DataFrame(raspredelenie, columns=['Unnamed: 15', 'Unnamed: 19'])
df['фамилия + имя'] = df['Unnamed: 15'].str.lower()
df = df.drop('Unnamed: 15', axis=1)
df = df.rename(columns={'Unnamed: 19': 'проект'})

df['Word Count'] = df['фамилия + имя'].str.split().str.len()
df_second = df[df['Word Count'] == 2].drop('Word Count', axis=1)

##

cur_raspredelenie = pd.concat([df_first, df_second], axis=0)
cur_raspredelenie['проект'] = cur_raspredelenie['проект'].astype(str)
cur_raspredelenie['фамилия + имя'] = cur_raspredelenie['фамилия + имя'].str.replace('ё', 'е').str.strip()

In [221]:
dict_with_raspredelenie = cur_raspredelenie.set_index('фамилия + имя')['проект'].to_dict()

In [222]:
cur_matches = matches.copy()

In [195]:
#dict_with_raspredelenie

In [227]:
for i in range(len(cur_matches)):
    print(cur_matches[i][0].name, cur_matches[i][0].partner.name)

хури полина 19
исламуратов марат 25
алейникова дарья 14
ревнивцев андрей 20
бурзунов данил 36
головач максим 48
нестерович светлана 45-46
мурзаева евгения 48
овчинников григорий 57
целищев максим 51
крылов артем 33
абазов руслан 24
анисимов максим 50
закиров дмитрий 23
иванов сергей 34-35
антонова дарья 34
хажиев роберт 28
макаров игорь 44
горбаренко анастасия 43
филистович илья 29
зубайдуллин ринат 47
вэлиулин вячеслав 31
кислицына полина 26
малинкович герман 32
запольская юлия 22
кузнецов дмитрий 37
мхитарян ваагн 25-26
касько константин 38
мусина айша 12
иванченко алена 30
шилоносова валентина 39
курченко лилия 40
тагиев станислав 42
юраков кирилл 56-57
борзунова алена 13
стержанова виталия 56
мировский слава 49
борзунова анастасия 17
кармастина яна 27
комаров олег 45
ипатов артем 18
фрыкина елена 54
берхольц виктория 53
хоменюк петр 55


In [197]:
for i in range(len(cur_matches)):
    cur_matches[i][1] = projects[dict_with_raspredelenie[cur_matches[i][0].name]]
    cur_matches[i][0].partner = projects[dict_with_raspredelenie[cur_matches[i][0].name]]
cur_matches

[[<__main__.Applicant at 0x1ac58bf8fa0>, <__main__.Project at 0x1ac5764dfc0>],
 [<__main__.Applicant at 0x1ac58be9d20>, <__main__.Project at 0x1ac5764d6c0>],
 [<__main__.Applicant at 0x1ac58bf9930>, <__main__.Project at 0x1ac5764ce20>],
 [<__main__.Applicant at 0x1ac58bf86a0>, <__main__.Project at 0x1ac5764dde0>],
 [<__main__.Applicant at 0x1ac574c8b80>, <__main__.Project at 0x1ac5764c700>],
 [<__main__.Applicant at 0x1ac574cae90>, <__main__.Project at 0x1ac58be8a30>],
 [<__main__.Applicant at 0x1ac58bf9180>, <__main__.Project at 0x1ac5764cf70>],
 [<__main__.Applicant at 0x1ac58bfaa40>, <__main__.Project at 0x1ac5764c490>],
 [<__main__.Applicant at 0x1ac58bfae90>, <__main__.Project at 0x1ac58be96f0>],
 [<__main__.Applicant at 0x1ac58bfb250>, <__main__.Project at 0x1ac5764df90>],
 [<__main__.Applicant at 0x1ac58bfa9e0>, <__main__.Project at 0x1ac58be8040>],
 [<__main__.Applicant at 0x1ac5764cac0>, <__main__.Project at 0x1ac5764c790>],
 [<__main__.Applicant at 0x1ac5764d120>, <__main__.P

In [201]:
[print(elem[0].name, elem[0].partner.name) for elem in cur_matches]

хури полина 12
исламуратов марат 34-35
алейникова дарья 49
ревнивцев андрей 20
бурзунов данил 15
головач максим 14
нестерович светлана 13
мурзаева евгения 48
овчинников григорий 19
целищев максим 45-46
крылов артем 25
абазов руслан 24
анисимов максим 22
закиров дмитрий 23
иванов сергей 34-35
антонова дарья 51
хажиев роберт 16
макаров игорь 33
горбаренко анастасия 45-46
филистович илья 42
зубайдуллин ринат 31
вэлиулин вячеслав 44
кислицына полина 18
малинкович герман 32
запольская юлия 29
кузнецов дмитрий 36
мхитарян ваагн 38
касько константин 47
мусина айша 43
иванченко алена 30
шилоносова валентина 39
курченко лилия 28
тагиев станислав 37
юраков кирилл 56-57
борзунова алена 40
стержанова виталия 48
мировский слава 50
борзунова анастасия 17
кармастина яна 27
комаров олег 55
ипатов артем 26
фрыкина елена 52
берхольц виктория 53
хоменюк петр 54


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [199]:
metrics(matches)

'среднее место проекта у студентов: 3.955, среднее место студентов у проектов: 22.720'

In [200]:
# # Создание студентов
#
# applicant1 = Applicant("Студент1", ["Проект1", "Проект2", "Проект3"])
# applicant2 = Applicant("Студент2", ["Проект2", "Проект1", "Проект3"])
# applicant3 = Applicant("Студент3", ["Проект3", "Проект2", "Проект1"])
# applicant4 = Applicant("Студент4", ["Проект3", "Проект2", "Проект1"])
# applicant5 = Applicant("Студент5", ["Проект2", "Проект3", "Проект1"])
#
# # Создание проекта
# school1 = Project("Проект1", 2, ["Студент1", "Студент2", "Студент3", "Студент4", "Студент5"])
# school2 = Project("Проект2", 1, ["Студент2", "Студент1", "Студент4", "Студент3", "Студент5"])
# school3 = Project("Проект3", 2, ["Студент2", "Студент4", "Студент3", "Студент1", "Студент5"])
#
# # Списки студентов и проектов
# applicants = [applicant1, applicant2, applicant3, applicant4, applicant5]
# schools = {"Проект1": school1, "Проект2": school2, "Проект3": school3}
#
# # Выполнение алгоритма Гейла-Шепли
# matches = gale_shapley(applicants, schools)
#
# matches